In [2]:
u = [2,2]
v = [2,3]
z = [3,5]

result = [sum(t) for t in zip(u,v,z)]
print(result)

[7, 10]


In [4]:
u = [1, 2, 3]
v = [4, 5, 6]
alpha = 2

result = [sum(t) for t in zip(u, v)]
print(result)

[5, 7, 9]


In [5]:
m1=[[3,6], [4,5]]
m2=[[5,8], [3,7]]

result = [[sum(row) for row in zip(*t)] for t in zip(m1, m2)]

print(result)

[[8, 14], [7, 12]]


In [8]:
mat=[[3,6], [4,5]]
a=4
result = [[a*element for element in t] for t in mat]

print(result)

[[12, 24], [16, 20]]


In [9]:
mat=[[1,2,3], [4,5,6]]
result=[[element for element in t] for t in zip(*mat)]

print(result)

[[1, 4], [2, 5], [3, 6]]


In [10]:
mat_a=[[1,1,2],[2,1,1]]
mat_b=[[1,1],[2,1],[1,3]]
result=[[sum(a*b for a,b in zip(row_a,col_b))
       for col_b in zip(*mat_b)] for row_a in mat_a]
print(result)

[[5, 8], [5, 6]]


In [11]:
import os

def get_file_list(dir_name): #os.listdir를 활용해 파일들을 가져옴
    return os.listdir(dir_name)

def get_conetents(file_list): #카테고리 분류
    y_class = []
    X_text = []
    class_dict = { # 0:야구, 1:축구
        1: "0", 2: "0", 3:"0", 4:"0", 5:"1", 6:"1", 7:"1", 8:"1"}
  

    for file_name in file_list:
        try:
            f = open(file_name, "r",  encoding="cp949")
            category = int(file_name.split(os.sep)[1].split("_")[0]) #파일명에서 카테고리 분리
            y_class.append(class_dict[category]) #파일을 0(야구),1(축구)로 분류
            X_text.append(f.read())
            f.close()
        except UnicodeDecodeError as e:
            print(e)
            print(file_name)
    return X_text, y_class



def get_cleaned_text(text): #의미없는 문장부호 삭제 & 소문자 변환
    import re
    text = re.sub('\W+','', text.lower() )
    return text


def get_corpus_dict(text):
    text = [sentence.split() for sentence in text] #이차원벡터
    cleaned_words = [get_cleaned_text(word) for words in text for word in words] #일차원벡터

    from collections import OrderedDict
    corpus_dict = OrderedDict()
    for i, v in enumerate(set(cleaned_words)):
        corpus_dict[v] = i #단어가 키값이 되게...
    return corpus_dict


def get_count_vector(text, corpus):#어떤 단어가 몇개 있는지
    text = [sentence.split() for sentence in text]
    word_number_list = [[corpus[get_cleaned_text(word)] for word in words] for words in text]
    X_vector = [[0 for _ in range(len(corpus))] for x in range(len(text))] #모두 0인 벡터

    for i, text in enumerate(word_number_list):
        for word_number in text:
            X_vector[i][word_number] += 1
    return X_vector

import math
def get_cosine_similarity(v1,v2):
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

def get_similarity_score(X_vector, source):
    source_vector = X_vector[source]
    similarity_list = []
    for target_vector in X_vector:
        similarity_list.append(
            get_cosine_similarity(source_vector, target_vector))
    return similarity_list


def get_top_n_similarity_news(similarity_score, n): 
    import operator
    x = {i:v for i, v in enumerate(similarity_score)}
    sorted_x = sorted(x.items(), key=operator.itemgetter(1))

    return list(reversed(sorted_x))[1:n+1]

def get_accuracy(similarity_list, y_class, source_news):
    source_class = y_class[source_news]

    return sum([source_class == y_class[i[0]] for i in similarity_list]) / len(similarity_list)


if __name__ == "__main__":
    dir_name = "news_data"
    file_list = get_file_list(dir_name)
    file_list = [os.path.join(dir_name, file_name) for file_name in file_list]
    X_text, y_class = get_conetents(file_list)

    corpus = get_corpus_dict(X_text)
    print("Number of words : {0}".format(len(corpus)))
    X_vector = get_count_vector(X_text, corpus)
    source_number = 10

    result = []

    for i in range(80):
        source_number = i

        similarity_score = get_similarity_score(X_vector, source_number)
        similarity_news = get_top_n_similarity_news(similarity_score, 10)
        accuracy_score = get_accuracy(similarity_news, y_class, source_number)
        result.append(accuracy_score)
    print(sum(result) / 80)

Number of words : 4024
0.6950000000000001
